# Spatially Assign Work
In this example, assignments will be assigned to specific workers based on the city district that it falls in. A layer in ArcGIS Online representing the city districts in Palm Springs will be used.

* Note: This example requires having Arcpy or Shapely installed in the Python environment.

### Import ArcGIS API for Python
Import the `arcgis` library and some modules within it.

In [1]:
from datetime import datetime
from arcgis.gis import GIS
from arcgis.geometry import Geometry
from arcgis.mapping import WebMap
from arcgis.apps import workforce
from datetime import datetime

/Users/nira9586/anaconda3/envs/workforce-scripts/lib/python3.7/site-packages/arcgis/features/_data/geodataset/utils.py:15: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  pd.datetime,


### Connect to organization and Get the Project
Let's connect to ArcGIS Online and get the Project with assignments.

In [2]:
gis = GIS("https://arcgis.com", "workforce_scripts")
item = gis.content.get("c765482bd0b9479b9104368da54df90d")
project = workforce.Project(item)

Enter password: ········


### Get Layer of City Districts
Let's get the layer representing city districts and display it.

In [3]:
districts_layer = gis.content.get("8a79535e0dc04410b5564c0e45428a2c").layers[0]
districts_map = gis.map("Palm Springs, CA", zoomlevel=10)
districts_map.add_layer(districts_layer)
districts_map

MapView(layout=Layout(height='400px', width='100%'), zoom=10.0)

### Add Assignments to the Map

In [4]:
districts_map.add_layer(project.assignments_layer)

### Create a spatially enabled dataframe of the districts

In [5]:
districts_df = districts_layer.query(as_df=True)

/Users/nira9586/anaconda3/envs/workforce-scripts/lib/python3.7/site-packages/arcgis/features/layer.py:1995: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  "esriFieldTypeDate" : pd.datetime,


### Get all of the unassigned assignments

In [6]:
assignments = project.assignments.search("status=0")

### Assign Assignments Based on Which District They Intersect¶
Let's fetch the districts layer and query to get all of the districts. Then, for each unassigned assignment intersect the assignment with all districts to determine which district it falls in. Assignments in district 10 should be assigned to James. Assignments in district 9 should be assigned to Aaron. Finally update all of the assignments using "batch_update".

In [7]:
aaron = project.workers.get(user_id="aaron_nitro")
james = project.workers.get(user_id="james_Nitro")
for assignment in assignments:
    contains = districts_df["SHAPE"].geom.contains(Geometry(assignment.geometry))
    containers = districts_df[contains]
    if not containers.empty: 
        district = containers['ID'].iloc[0]
        if district == 10:
            assignment.worker = james
            assignment.status = "assigned"
            assignment.assigned_date = datetime.utcnow()
        elif district == 9:
            assignment.worker = aaron
            assignment.status = "assigned"
            assignment.assigned_date = datetime.utcnow()
assignments = project.assignments.batch_update(assignments)


### Verify Assignments are Assigned

In [8]:
webmap = gis.map("Palm Springs", zoomlevel=11)
webmap.add_layer(project.assignments_layer)
webmap

MapView(layout=Layout(height='400px', width='100%'), zoom=11.0)